In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
from ta.volatility import BollingerBands
from ta.trend import MACD
from ta.momentum import RSIIndicator

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

In [5]:
df = pd.read_csv('XAUUSD.csv')
df

,time,open,high,low,close,tick_volume
0,2016-12-15 02:00:00,1142.34,1144.47,1142.06,1143.62,9861
1,2016-12-15 03:00:00,1143.58,1143.78,1141.58,1142.57,11524
2,2016-12-15 04:00:00,1142.59,1144.03,1140.87,1141.88,14523
3,2016-12-15 05:00:00,1141.88,1142.20,1140.90,1141.45,10728
4,2016-12-15 06:00:00,1141.45,1143.17,1140.69,1141.19,12983
...,...,...,...,...,...,...
37838,2023-06-21 06:00:00,1935.49,1935.65,1931.21,1934.93,5773
37839,2023-06-21 07:00:00,1934.93,1937.40,1933.84,1935.00,5108
37840,2023-06-21 08:00:00,1935.00,1935.92,1933.61,1934.61,4420
37841,2023-06-21 09:00:00,1934.63,1935.87,1932.10,1933.29,4503


In [4]:
# device agnostic code.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# Define the sequence length
sequence_length = 50 

# Normalization
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data.values[:, 1:]) 

# Convert pandas DataFrame to Tensor
data_normalized = torch.FloatTensor(data_normalized).to(device)

# Creating sequences
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1, data.columns.get_loc('close') - 1]  # Adjusted for 0-indexing
        inout_seq.append((train_seq ,train_label))
    return inout_seq

train_size = int(len(data_normalized) * 0.7)
train_data = data_normalized[:train_size]
test_data = data_normalized[train_size:]

train_sequence = create_inout_sequences(train_data, sequence_length)
test_sequence = create_inout_sequences(test_data, sequence_length)



In [8]:
# Creating dataloaders
batch_size = 64  # Feel free to adjust this

train_dataloader = DataLoader(train_sequence, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_sequence, batch_size=batch_size, shuffle=False)


In [23]:
# Create LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=11, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size).to(device),
                            torch.zeros(1,1,self.hidden_layer_size).to(device))
    def init_hidden(self, batch_size):
        
        return (torch.zeros(1, batch_size, self.hidden_layer_size).to(device),
            torch.zeros(1, batch_size, self.hidden_layer_size).to(device))

    def forward(self, input_seq):

        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions.view(input_seq.size(0), -1)



# Instantiate the model, define the loss and the optimizer
model = LSTM(input_size = 11).to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
# device agnostic code.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
# Training loop
epochs = 150

for i in range(epochs):
    for seq, labels in train_dataloader:
        optimizer.zero_grad()
        model.hidden_cell = model.init_hidden(seq.size(0))  # initialize with batch size

        y_pred = model(seq.to(device))

        single_loss = loss_function(y_pred, labels.to(device))
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

RuntimeError: input.size(-1) must be equal to input_size. Expected 11, got 550